In [2]:
import numpy as np
import networkx as nx
import random
import pandas as pd
import csv
import matplotlib.pyplot as plt
import itertools


ID1, ID2 = 319044434, 314779166

NoseBook_path = 'NoseBook_friendships.csv'
cost_path = 'costs.csv'


def influencers_submission(ID1, ID2, lst):
    with open(f'{ID1}_{ID2}.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(lst)


def create_graph(edges_path: str) -> nx.Graph:
    """
    Creates the NoseBook social network
    :param edges_path: A csv file that contains information obout "friendships" in the network
    :return: The NoseBook social network
    """
    edges = pd.read_csv(edges_path).to_numpy()
    net = nx.Graph()
    net.add_edges_from(edges)
    return net


def buy_products(net: nx.Graph, purchased: set) -> set:
    """
    Gets the network at the beginning of stage t and simulates a purchase round
    :param net: The network at stage t
    :param purchased: All the users who recieved or bought the product up to and including stage t-1
    :return: All the users who recieved or bought the product up to and including stage t
    """
    new_purchases = set()
    for user in net.nodes:
        neighborhood = set(net.neighbors(user))
        b = len(neighborhood.intersection(purchased))
        n = len(neighborhood)
        prob = b / n
        if prob >= random.uniform(0, 1):
            new_purchases.add(user)

    return new_purchases.union(purchased)


def product_exposure_score(net: nx.Graph, purchased_set: set) -> int:
    """
    Returns the number of users who have seen the product
    :param purchased_set: A set of users who bought the product
    :param net: The NoseBook social network
    :return:  The sum for all users who have seen the product.
    """
    exposure = 0
    for user in net.nodes:
        neighborhood = set(net.neighbors(user))

        if user in purchased_set:
            exposure += 1
        elif len(neighborhood.intersection(purchased_set)) != 0:
            b = len(neighborhood.intersection(purchased_set))
            rand = random.uniform(0, 1)
            if rand < 1 / (1 + 10 * np.exp(-b/2)):
                exposure += 1
    return exposure


def get_influencers_cost(cost_path: str, influencers: list) -> int:
    """
    Returns the cost of the influencers you chose
    :param cost_path: A csv file containing the information about the costs
    :param influencers: A list of your influencers
    :return: Sum of costs
    """
    costs = pd.read_csv(cost_path)
    return sum([costs[costs['user'] == influencer]['cost'].item() if influencer in list(costs['user']) else 0 for influencer in influencers])


""" ~ Our functions ~ """

class Node:
    def __init__(self, id, num_neighbors, expected_num_bought_neighbors=[0]*6):
        self.id = id
        self.cost = None
        self.num_neighbors = num_neighbors
        self.expected_num_bought_neighbors = expected_num_bought_neighbors
    
    def get_buying_prob(self, t):
        """ Returns the probability of buying at round t """
        return self.expected_num_bought_neighbors[t] / self.num_neighbors
    
    def add_neighbor_to_buyers(self, t):
        """ Adds a neighbor to the set of neighbors that bought at round t """
        self.expected_num_bought_neighbors[t] += 1

    def set_cost(self, cost):
        self.cost = cost


def choose_influencers(NoseBook_network, ordered_nodes):
    """
    Choose the influencers we want to start with
    :param NoseBook_network: The NoseBook social network
    :param ordered_nodes: A list of the nodes ordered by a centrality measure of some sort
    :return: A list of the influencers we chose
    """
    # the total cost of the influencers should not exceed 1000
    money_left = 1000
    influencers = []
    neigh_to_remove = set()
    i = 0
    while money_left > 0 and i < len(ordered_nodes):
        influencer = ordered_nodes[i]
        i += 1
        # If the influencer is too expensive, skip to the next one
        while get_influencers_cost(cost_path, influencers + [influencer]) > 1000:
            influencer = ordered_nodes[i]
            i += 1
        influencers.append(influencer)

        curr_neighbors = {influencer}
        nodes_created = dict()
        # Simulating 6 rounds of buying
        for t in range(6):
            neighbors = set()
            curr_neighbors = curr_neighbors.difference(neigh_to_remove)  # Remove the neighbors that already bought in a previous round from the neighbors of the current round
            neigh_to_remove = neigh_to_remove.union(curr_neighbors)  # Add the neighbors that already bought in a previous round to the set of neighbors to remove

            # Update the neighbors of the current neighbors
            for neigh in curr_neighbors:
                curr_node_neighs = set(NoseBook_network.neighbors(neigh))
                neighbors = neighbors.union(curr_node_neighs)  # Add the neighbors of the current neighbors
                # Create a node for each neighbor
                if neigh not in nodes_created.keys():
                    node = Node(neigh, len(curr_node_neighs))
                    nodes_created[neigh] = node
                else:
                    node = nodes_created[neigh]
                    node.add_neighbor_to_buyers(t)

            curr_neighbors = neighbors  # Add bernoulli according to the probability of buying
                                        # Also, cancel duplicate detection and refer to probabilities


        ordered_nodes = [node for node in ordered_nodes if node not in neigh_to_remove]
        money_left -= get_influencers_cost(cost_path, influencers)  # Update the money left after the addition of the influencer
    return influencers


def get_influencers_by_score(sorted_nodes, costs):
    """ Get the influencers based on the score and the cost.
        This function is instead of the choose_influencers function. """
    budget = 1000
    influencers = []
    current_budget = budget
    for node in sorted_nodes:
        if costs[node] <= current_budget:  # If the cost of the influencer is less than the budget, add them
            influencers.append(node)
            current_budget -= costs[node]  # Update the budget
        if current_budget <= 0:
            break    
    return influencers


def calculate_neighborhood_overlap(graph, node_a, node_b):
    """ Calculate the neighborhood overlap between two nodes """
    neighbors_a = set(graph.neighbors(node_a))
    neighbors_b = set(graph.neighbors(node_b))
    
    common_neighbors = neighbors_a.intersection(neighbors_b)  # Intersection of neighbors
    all_neighbors = neighbors_a.union(neighbors_b)  # Union of neighbors
    
    if len(all_neighbors) == 0:  # Avoiding division by zero
        return 0
    
    # Neighborhood overlap calculation
    overlap = len(common_neighbors) / len(all_neighbors)
    return overlap


def visualize_influence_by_cost(nodes, centrality_measure, costs, title):
    """ Visualize the influence of the nodes by their cost """
    degree = [centrality_measure[node] for node in nodes]
    cost = [costs[node] for node in nodes]
    plt.scatter(cost, degree)
    plt.xlabel('Cost')
    plt.ylabel(title)
    plt.title('score by Cost')
    plt.show()


def get_expectation_of_final_score(NoseBook_network, purchased):
    """ Get the expectation of the final score """
    epochs = 20  # Number of epochs to run, change this value
    scores = np.zeros(epochs)
    initial_purchased = purchased  # Save the initial purchased set
    for j in range(epochs):
        # Run the simulation for 6 rounds
        for i in range(6):
            purchased = buy_products(NoseBook_network, purchased)  # Find the new nodes that bought the product

        score = product_exposure_score(NoseBook_network, purchased)  # Calculate the score
        purchased = initial_purchased
        scores[j] = score
    print("*************** Expected final score is " + str(np.mean(scores)) + " ***************")  # Print the approximation to the expectation of the final score
    return np.mean(scores)

def most_neighbors_per_cost(centrality_measure, costs):
    """ Get the 5 nodes with the best value w.r.t a centrality measure per cost """
    k = 10 # Number of nodes to choose, change this value
    costs_set = set(costs.values())
    # A dictionary that will contain all nodes with the same cost
    dict_of_costs = {cost: [] for cost in costs_set}
    for cost in costs_set:
        # Making lists of nodes with the same cost
        dict_of_costs[cost] = [node for node in centrality_measure.keys() if costs[node] == cost]
    # The 5 nodes with the best value w.r.t a centrality measure per cost
    top_k_per_cost = {cost: sorted(dict_of_costs[cost], key=lambda x: centrality_measure[x], reverse=True)[:k] for cost in costs_set}

    # Print the chosen k nodes for each cost
    for cost in costs_set:
        print("Cost: ", cost, str(k)+"  top nodes: ")
        for node in top_k_per_cost[cost]:
            print(node, " Centrality measure: ", centrality_measure[node])
    return top_k_per_cost


def build_neighborhood_overlap_matrix(graph, top_k_per_cost):
    """ Build the neighborhood overlap matrix, only for the top k nodes (per cost). """
    top_nodes = [node for nodes in top_k_per_cost.values() for node in nodes]

    # Create a mapping of nodes to indices
    node_to_index = {node: i for i, node in enumerate(top_nodes)}

    # Calculate and store neighborhood overlap for every pair of nodes
    num_nodes = len(top_nodes)
    overlap_matrix = [[0 for _ in range(num_nodes)] for _ in range(num_nodes)]
    for node_a in top_nodes:
        for node_b in top_nodes:
            overlap = calculate_neighborhood_overlap(graph, node_a, node_b)
            index_a = node_to_index[node_a]
            index_b = node_to_index[node_b]
            # Store the overlap in both symmetric positions in the matrix
            overlap_matrix[index_a][index_b] = overlap
            overlap_matrix[index_b][index_a] = overlap
            print("Overlap between nodes", node_a, "and", node_b, "is", overlap)
    # overlap_matrix now contains the neighborhood overlap values for all node pairs
    return overlap_matrix, node_to_index


def scores_calculations(NoseBook_network, top_k_per_cost):
    """ Calculate the scores for the top k nodes per cost """
    # Normalize and compute cost-effectiveness
    degree_scores = {node: costs[node] ** (degree_centrality[node] * 10) if node in costs and costs[node] != 0 else 0 for node in NoseBook_network.nodes}

    # closeness_scores = {node: betweenness_centrality[node] * 10 + 100 * costs[node]  if node in costs and costs[node] != 0 else 0 for node in NoseBook_network.nodes}
    # neighborhood_overlap_scores = {node: (sum(overlap_matrix[node_to_index[node]]) / num_nodes) * 10 + 100 * costs[node] if node in costs and costs[node] != 0 else 0 for node in NoseBook_network.nodes}
    # Sort nodes based on cost-effectiveness
    sorted_by_degree = sorted(degree_scores.keys(), key=lambda x: degree_scores[x], reverse=True)
    
    # sorted_by_closeness = sorted(closeness_scores.keys(), key=lambda x: closeness_scores[x], reverse=True)
    # sorted_by_neighborhood_overlap = sorted(neighborhood_overlap_scores.keys(), key=lambda x: neighborhood_overlap_scores[x], reverse=False)
    return sorted_by_degree


def normalize_and_combine_centrality_scores(*centrality_dicts):
    # Extract nodes
    nodes = list(centrality_dicts[0].keys())
    
    # Combine all centrality scores into a single array
    combined_scores = np.array([
        [centrality_dict[node] for centrality_dict in centrality_dicts]
        for node in nodes
    ])
    
    # Normalize scores manually
    min_vals = combined_scores.min(axis=0)
    max_vals = combined_scores.max(axis=0)
    normalized_scores = (combined_scores - min_vals) / (max_vals - min_vals)
    
    # Sum normalized scores to get a single combined score for each node
    combined_scores = normalized_scores.sum(axis=1)
    
    # Create a dictionary of combined scores
    combined_centrality = dict(zip(nodes, combined_scores))
    
    return combined_centrality




if __name__ == '__main__':

    print("STARTING")

    NoseBook_network = create_graph(NoseBook_path)
    # # Draw the network
    # pos = nx.random_layout(NoseBook_network)  # positions for all nodes
    # nx.draw(NoseBook_network, pos, with_labels=False, node_size=10, width=0.01, alpha=0.5, node_color='teal')
    # plt.show()
    
    costs = pd.read_csv(cost_path)
    costs = dict(zip(costs['user'], costs['cost']))

    degree_centrality = nx.degree_centrality(NoseBook_network)
    closeness_centrality = nx.closeness_centrality(NoseBook_network)  # Runs for too long
    betweenness_centrality = nx.betweenness_centrality(NoseBook_network)
    eigen_centrality = nx.eigenvector_centrality(NoseBook_network)
    #page_rank = nx.pagerank(NoseBook_network)


    """CURRENTLY WORKS THE BEST"""
    # visualize_influence_by_cost(NoseBook_network.nodes, degree_centrality, costs," degree_rank")#-currently works the best
    # visualize_influence_by_cost(degree_centrality,closeness_centrality, costs, "clossness_rank")
    # visualize_influence_by_cost(degree_centrality, eigen_centrality, costs,'eigen_rank')
    # #top_5_per_cost = most_neighbors_per_cost(degree_centrality, costs) - currently works best

   # combined_centrality = normalize_and_combine_centrality_scores(degree_centrality, closeness_centrality, betweenness_centrality, eigen_centrality)

    
    # comb_scores = {node: 100*degree_centrality[node]+costs[node] if node in costs and costs[node] != 0 else 0 for node in NoseBook_network.nodes}
    # # Sort nodes by normalized degree centrality
    # sorted_nodes = sorted(comb_scores, key=lambda x: comb_scores[x], reverse=True)
    # print("Sorted nodes: ", sorted_nodes)
    # #influencers= get_influencers_by_score(sorted_nodes, costs)
    # #influencers = choose_influencers(NoseBook_network, list(sorted_nodes))
    # influencers=[2035]
    # print("Influencers: ", influencers)
    # influencers_cost = get_influencers_cost(cost_path, influencers)
    # print("Influencers cost: ", influencers_cost)
    # if influencers_cost > 1000:
    #     print("*************** Influencers are too expensive! ***************")
    #     exit()
    # purchased = set(influencers)    
    # get_expectation_of_final_score(NoseBook_network, purchased)


  #  print("Combined centrality: ", combined_centrality)

    """trying to find the best score - each top 10 nodes per cost"""
#    # visualize_influence_by_cost(NoseBook_network.nodes, betweenness_centrality, costs)
    top_30_per_cost_deg = most_neighbors_per_cost(degree_centrality, costs) 
    top_30_per_cost_bet = most_neighbors_per_cost(betweenness_centrality, costs) 
    top_30_per_cost_clos = most_neighbors_per_cost(closeness_centrality, costs)
    top_30_per_cost_eig = most_neighbors_per_cost(eigen_centrality, costs)
    # #intersection of the four
    # top_5_per_cost = {cost: list(set(top_5_per_cost_deg[cost]) & set(top_5_per_cost_bet[cost]) & set(top_5_per_cost_clos[cost]) & set(top_5_per_cost_eig[cost])) for cost in costs.values()}
    # print("Top 5 per cost: ", top_5_per_cost)
    # scores_calculations(NoseBook_network, top_5_per_cost)

    # top_5_per_cost_combined = most_neighbors_per_cost(combined_centrality, costs)
    # print("Top 5 per cost combined: ", top_5_per_cost_combined)

    # influencers = get_influencers_by_score(sorted_by_degree, costs)
    """ Current best score - influencers = [1608,3266,3260,3448]-1460 """
    #shuffle the nodes to make every run different  [1608,3266,3260,3448]
    list_100=set(top_30_per_cost_bet[100],top_30_per_cost_deg[100],top_30_per_cost_clos[100],top_30_per_cost_eig[100])
    # calculate matrix of neighborhood overlap of the list_100 nodes
    overlap_matrix = build_neighborhood_overlap_matrix(NoseBook_network, list_100)
    print(overlap_matrix)






    # Generate all permutations and store in a list
    # all_permutations = list(itertools.permutations(nodes))
    # random.shuffle(all_permutations)
    
    # Print shuffled permutations
# for perm in all_permutations:
#     influencers = list(perm)
    # print("Influencers: ", influencers)
    # influencers_cost = get_influencers_cost(cost_path, influencers)
    # print("Influencers cost: ", influencers_cost)
    # if influencers_cost > 1000:
    #     print("*************** Influencers are too expensive! ***************")
    #     exit()
    # # purchased = set(influencers)    
    # get_expectation_of_final_score(NoseBook_network, purchased)
    list_500= [3299, 1478, 1608, 2255, 2389]   
    list_400 = [1090, 1091, 3588, 2341, 1769, 975, 3024, 3439, 2647, 3961, 506, 796, 3454]

#     '''maximization by trying the best nodes with 100 score that did not make through the intersection of each centrality'''
#     nodes_cost_100_set1 = [
#         318, 2295, 617, 3002, 1455, 3448, 1453, 814, 2480, 3391
#     ]
#     nodes_cost_100_set2 = [
#         3892, 2516, 3798, 3654, 200, 3448, 2191, 3293, 1897, 1887
#     ]
#     nodes_cost_100_set3 = [
#         3798, 3654, 1897, 2771, 3659, 1719, 2282, 3047, 1110
#     ]
#     nodes_cost_100_set4 = [
#         318, 617, 2295, 3002, 2480, 814, 1453, 259, 3391, 525
#     ]
#     node_cost_100_set5 =[318,2295, 617, 3002,1455,3448,1453,814, 2480,3391 ]

#     # Combine all sets into a single list
#     all_nodes = (
#         nodes_cost_100_set1 + nodes_cost_100_set2 +
#         nodes_cost_100_set3 + nodes_cost_100_set4 + node_cost_100_set5)
    
#     #influencers = choose_influencers(NoseBook_network, list(sorted_nodes))
#     print("Influencers: ", influencers)
#     influencers_cost = get_influencers_cost(cost_path, influencers)
#     print("Influencers cost: ", influencers_cost)
#     if influencers_cost > 1000:
#         print("*************** Influencers are too expensive! ***************")
#         exit()
#     purchased = set(influencers)    
#     get_expectation_of_final_score(NoseBook_network, purchased)

#     # Remove duplicates by converting the list to a set and back to a list
#    # all_nodes= [1090, 1091, 3588, 2341, 1769, 975, 3024, 3439, 2647, 3961, 506, 796, 3454]
#     unique_nodes = list(set(all_nodes))
#     print("Unique nodes: ", unique_nodes)

  
   
    exp_scores = []

    # Calculate expectation scores for each nod
    # for node1 in unique_nodes:
    # #     influencers.append(node1)
    # influencers=[]
    # for node_500 in list_500:
    #     influencers.append(node_500)
    #     for node_400 in list_400:
    #         influencers.append(node_400)
    #         for node in unique_nodes:
    #             influencers.append(node)
    #             purchased = set(influencers)
    #             print("Influencers: ", influencers)
    #             influencers_cost = get_influencers_cost(cost_path, influencers)
    #             print("Influencers cost: ", influencers_cost)
    #             exp_score = get_expectation_of_final_score(NoseBook_network, purchased)
               
                
    #             # Append the score only if it is not None
    #             if exp_score is not None:
    #                 exp_scores.append((node, exp_score))
    #             influencers.remove(node)    
    #         # influencers.remove(node1)
    #         # Find the maximum expectation score and corresponding node
    #         influencers.remove(node_400)
    #     influencers.remove(node_500)    
    # if exp_scores:
    #     max_node, max_score = max(exp_scores, key=lambda x: x[1])
    #     # Print the result
    #     print("*************** Your final score is " + str(max_score) + ", node " + str(max_node) + " ***************")
    # else:
    #     print("No valid scores calculated.")



'''the original main'''

    # for i in range(6):
    #     purchased = buy_products(NoseBook_network, purchased)
    #     print("finished round", i + 1)

    # score = product_exposure_score(NoseBook_network, purchased)

    # print("*************** Your final score is " + str(score) + " ***************")

# iterate over these lists with permutations
list_500= [3299, 1478, 1608, 2255, 2389]   
list_400 = [1090, 1091, 3588, 2341, 1769, 975, 3024, 3439, 2647, 3961, 506, 796, 3454]





STARTING
Cost:  800 10  top nodes: 
2035  Centrality measure:  0.1961367013372957
Cost:  100 10  top nodes: 
318  Centrality measure:  0.045567112431896976
2295  Centrality measure:  0.04432887568103021
617  Centrality measure:  0.044081228330856856
3002  Centrality measure:  0.04358593363051015
1455  Centrality measure:  0.042842991579990095
3448  Centrality measure:  0.04210004952947003
1453  Centrality measure:  0.04185240217929668
814  Centrality measure:  0.041604754829123326
2480  Centrality measure:  0.041604754829123326
3391  Centrality measure:  0.041604754829123326
Cost:  200 10  top nodes: 
3266  Centrality measure:  0.05794947994056463
3260  Centrality measure:  0.05671124318969787
677  Centrality measure:  0.0554730064388311
855  Centrality measure:  0.0549777117384844
2686  Centrality measure:  0.054730064388311045
2752  Centrality measure:  0.052005943536404156
786  Centrality measure:  0.0512630014858841
2628  Centrality measure:  0.0512630014858841
2796  Centrality mea

TypeError: set expected at most 1 argument, got 4

In [3]:
def get_expectation_of_final_score(NoseBook_network, purchased):
    """ Get the expectation of the final score """
    epochs = 100  # Number of epochs to run, change this value
    scores = np.zeros(epochs)
    initial_purchased = purchased  # Save the initial purchased set
    for j in range(epochs):
        # Run the simulation for 6 rounds
        for i in range(6):
            purchased = buy_products(NoseBook_network, purchased)  # Find the new nodes that bought the product

        score = product_exposure_score(NoseBook_network, purchased)  # Calculate the score
        purchased = initial_purchased
        scores[j] = score
    print("*************** Expected final score is " + str(np.mean(scores)) + " ***************")  # Print the approximation to the expectation of the final score
    return np.mean(scores)

In [4]:
def most_neighbors_per_cost(centrality_measure, costs):
    """ Get the 5 nodes with the best value w.r.t a centrality measure per cost """
    k = 30 # Number of nodes to choose, change this value
    costs_set = set(costs.values())
    # A dictionary that will contain all nodes with the same cost
    dict_of_costs = {cost: [] for cost in costs_set}
    for cost in costs_set:
        # Making lists of nodes with the same cost
        dict_of_costs[cost] = [node for node in centrality_measure.keys() if costs[node] == cost]
    # The 5 nodes with the best value w.r.t a centrality measure per cost
    top_k_per_cost = {cost: sorted(dict_of_costs[cost], key=lambda x: centrality_measure[x], reverse=True)[:k] for cost in costs_set}

    # Print the chosen k nodes for each cost
    for cost in costs_set:
        print("Cost: ", cost, str(k)+"  top nodes: ")
        for node in top_k_per_cost[cost]:
            print(node, " Centrality measure: ", centrality_measure[node])
    return top_k_per_cost

In [5]:
top_30_per_cost_deg = most_neighbors_per_cost(degree_centrality, costs) 
top_30_per_cost_bet = most_neighbors_per_cost(betweenness_centrality, costs) 
top_30_per_cost_clos = most_neighbors_per_cost(closeness_centrality, costs)
top_30_per_cost_eig = most_neighbors_per_cost(eigen_centrality, costs)

Cost:  800 30  top nodes: 
2035  Centrality measure:  0.1961367013372957
Cost:  100 30  top nodes: 
318  Centrality measure:  0.045567112431896976
2295  Centrality measure:  0.04432887568103021
617  Centrality measure:  0.044081228330856856
3002  Centrality measure:  0.04358593363051015
1455  Centrality measure:  0.042842991579990095
3448  Centrality measure:  0.04210004952947003
1453  Centrality measure:  0.04185240217929668
814  Centrality measure:  0.041604754829123326
2480  Centrality measure:  0.041604754829123326
3391  Centrality measure:  0.041604754829123326
525  Centrality measure:  0.041604754829123326
259  Centrality measure:  0.041357107478949974
3115  Centrality measure:  0.04086181277860327
135  Centrality measure:  0.04061416542842991
2679  Centrality measure:  0.040118870728083206
2217  Centrality measure:  0.040118870728083206
932  Centrality measure:  0.039871223377909853
2154  Centrality measure:  0.039871223377909853
3995  Centrality measure:  0.039871223377909853
1

In [6]:
all_120= (top_30_per_cost_bet[120]+top_30_per_cost_deg[120]+top_30_per_cost_clos[120]+top_30_per_cost_eig[120])
all_120=list(set(all_120))
print(all_120)
all_200= (top_30_per_cost_bet[200]+top_30_per_cost_deg[200]+top_30_per_cost_clos[200]+top_30_per_cost_eig[200])
all_200=list(set(all_200))
print(all_200)
all_120_200= all_120+all_200
all_120_200=list(set(all_120_200))
print(len(all_120_200))

[6, 526, 1041, 1059, 2091, 2608, 3121, 1084, 2624, 2118, 3154, 2642, 595, 3172, 2662, 1127, 3695, 116, 2687, 645, 1672, 1167, 2201, 3234, 3236, 2218, 690, 1207, 1211, 3269, 2767, 1761, 3818, 2283, 1260, 757, 1287, 3338, 2834, 3355, 1314, 1841, 307, 821, 309, 3385, 838, 3410, 339, 349, 3422, 1396, 2949, 1414, 2953, 3978, 396, 2444, 3984, 1945, 1440, 942, 3503, 4019, 4023, 4024, 1476, 3526, 475, 2523, 3035, 1504, 1512, 489, 3563, 3564, 3577]
[3584, 3975, 2056, 2574, 146, 786, 2460, 160, 34, 164, 676, 3877, 677, 2088, 1960, 3370, 2468, 1448, 687, 3120, 1840, 1970, 818, 1076, 566, 1767, 3260, 2752, 833, 3266, 2628, 455, 2120, 331, 77, 1746, 2003, 2898, 2515, 2006, 3031, 600, 855, 1494, 345, 2781, 990, 2527, 864, 224, 1117, 3940, 2405, 2149, 1511, 2280, 361, 743, 619, 1644, 2796, 1388, 1391, 624, 2544, 496, 2542, 627, 247, 2686]
147


In [7]:
#list_100=[top_30_per_cost_bet[100]]+[top_30_per_cost_deg[100]]+[top_30_per_cost_clos[100]]+[top_30_per_cost_eig[100]]
# calculate matrix of neighborhood overlap of the list_100 nodes
# overlap_matrix = build_neighborhood_overlap_matrix(NoseBook_network, list_100)
# print(overlap_matrix)
#make unique nodes
all_nodes = (
        top_30_per_cost_deg[100] + top_30_per_cost_bet[100] +
        top_30_per_cost_clos[100] + top_30_per_cost_eig[100])
unique_nodes = list(set(all_nodes))
print("Unique nodes: ", sorted(unique_nodes))
print("len unique nodes", len(unique_nodes))

Unique nodes:  [24, 81, 132, 135, 200, 229, 259, 291, 317, 318, 319, 350, 367, 384, 449, 511, 525, 617, 707, 729, 746, 777, 798, 814, 872, 932, 1080, 1110, 1259, 1264, 1274, 1421, 1453, 1455, 1483, 1589, 1651, 1719, 1817, 1887, 1897, 2154, 2191, 2217, 2245, 2282, 2295, 2394, 2412, 2480, 2503, 2516, 2576, 2583, 2623, 2679, 2771, 2881, 2972, 3002, 3047, 3112, 3115, 3125, 3293, 3303, 3318, 3391, 3448, 3461, 3557, 3642, 3654, 3659, 3681, 3731, 3792, 3798, 3802, 3851, 3892, 3926, 3995]
len unique nodes 83


In [ ]:
#
import networkx as nx
import numpy as np

def neighborhood_overlap(graph, node1, node2):
    if node1 == node2:
        return 1
    neighbors1 = set(graph.neighbors(node1))
    neighbors2 = set(graph.neighbors(node2))
    
    intersection = len(neighbors1 & neighbors2)
    union = len(neighbors1 | neighbors2)
    
    if union == 0:
        return 0
    else:
        return intersection / union

def create_neighborhood_overlap_matrix(graph, nodes):
    size = len(nodes)
    overlap_matrix = np.zeros((size, size))
    
    for i in range(size):
        for j in range(size):
            overlap_matrix[i][j] = neighborhood_overlap(graph, nodes[i], nodes[j])
    
    return overlap_matrix



In [ ]:
overlap_matrix = create_neighborhood_overlap_matrix(NoseBook_network, unique_nodes)
#make the matrix data frame
overlap_matrix_df = pd.DataFrame(overlap_matrix, columns=unique_nodes, index=unique_nodes)
#show the data frame
print(overlap_matrix_df)

          259       525       2191      1453      814       1455      2480  \
259   1.000000  0.700508  0.000000  0.759162  0.753927  0.666667  0.717949   
525   0.700508  1.000000  0.000000  0.737113  0.740933  0.647343  0.740933   
2191  0.000000  0.000000  1.000000  0.000000  0.000000  0.000000  0.000000   
1453  0.759162  0.737113  0.000000  1.000000  0.764398  0.676471  0.746114   
814   0.753927  0.740933  0.000000  0.764398  1.000000  0.623810  0.759162   
1455  0.666667  0.647343  0.000000  0.676471  0.623810  1.000000  0.631579   
2480  0.717949  0.740933  0.000000  0.746114  0.759162  0.631579  1.000000   
3892  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
1719  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
3002  0.723618  0.728643  0.000000  0.725000  0.720000  0.694175  0.746193   
318   0.763819  0.692308  0.000000  0.782828  0.751244  0.668224  0.760000   
3391  0.700508  0.731959  0.000000  0.693467  0.759162  0.631579

In [ ]:
#export data frame to csv
overlap_matrix_df.to_csv('overlap_matrix.csv')
#open with excel
overlap_matrix_df.to_excel('overlap_matrix.xlsx')


In [ ]:


def is_valid_group(group, overlap_matrix):
    for i in range(len(group)):
        for j in range(i + 1, len(group)):
            if overlap_matrix[group[i]][group[j]] != 0:
                return False
    return True

def find_groups(overlap_matrix, group_size=8, start_index=0, current_group=None, all_groups=None):
    if current_group is None:
        current_group = []
    if all_groups is None:
        all_groups = []
    
    if len(current_group) == group_size:
        if is_valid_group(current_group, overlap_matrix):
            all_groups.append(current_group.copy())
        return all_groups
    
    for i in range(start_index, len(overlap_matrix)):
        current_group.append(i)
        find_groups(overlap_matrix, group_size, i + 1, current_group, all_groups)
        current_group.pop()
      
    return all_groups

In [ ]:
valid_groups = find_groups(overlap_matrix)
print(valid_groups)

[]


In [ ]:
#I took it single handedly
influencers=[259,3461,3851,2191,3293,3448, 1080]
print("Influencers: ", influencers)
influencers_cost = get_influencers_cost(cost_path, influencers)
print("Influencers cost: ", influencers_cost)
if influencers_cost > 1000:
    print("*************** Influencers are too expensive! ***************")
    exit()
purchased = set(influencers)    
get_expectation_of_final_score(NoseBook_network, purchased)

Influencers:  [259, 3461, 3851, 2191, 3293, 3448, 1080]
Influencers cost:  700
*************** Expected final score is 930.3 ***************


930.3

In [ ]:
unique_nodes2 = set(unique_nodes) - set([259, 3461, 3851, 2191, 3293, 3448, 1080])
print(unique_nodes2)


{135, 777, 525, 3731, 24, 3995, 291, 932, 2679, 2217, 3115, 1453, 814, 1455, 2480, 3892, 1589, 3125, 1719, 3002, 3642, 318, 3391, 319, 449, 707, 3654, 2503, 200, 3659, 3792, 2771, 2516, 3798, 1110, 729, 2394, 350, 1887, 3047, 872, 617, 2154, 1897, 2282, 367, 3318, 2295}


In [ ]:

for node_500 in list_500:
        influencers.append(node_500)
        for node_400 in list_400:
            influencers.append(node_400)
            for node in unique_nodes:
                influencers.append(node)
                purchased = set(influencers)
                print("Influencers: ", influencers)
                influencers_cost = get_influencers_cost(cost_path, influencers)
                print("Influencers cost: ", influencers_cost)
                exp_score = get_expectation_of_final_score(NoseBook_network, purchased)
               
                
                # Append the score only if it is not None
                if exp_score is not None:
                    exp_scores.append((node, exp_score))
                influencers.remove(node)    
            # influencers.remove(node1)
            # Find the maximum expectation score and corresponding node
            influencers.remove(node_400)
        influencers.remove(node_500)    
    if exp_scores:
        max_node, max_score = max(exp_scores, key=lambda x: x[1])
        # Print the result
        print("*************** Your final score is " + str(max_score) + ", node " + str(max_node) + " ***************")
    else:
        print("No valid scores calculated.")


In [8]:
all_120= (top_30_per_cost_bet[120]+top_30_per_cost_deg[120]+top_30_per_cost_clos[120]+top_30_per_cost_eig[120])
all_120=list(set(all_120))
print(all_120)
all_200= (top_30_per_cost_bet[200]+top_30_per_cost_deg[200]+top_30_per_cost_clos[200]+top_30_per_cost_eig[200])
all_200=list(set(all_200))
print(all_200)
all_120_200= all_120+all_200
all_120_200=list(set(all_120_200))
print(len(all_120_200))

[6, 526, 1041, 1059, 2091, 2608, 3121, 1084, 2624, 2118, 3154, 2642, 595, 3172, 2662, 1127, 3695, 116, 2687, 645, 1672, 1167, 2201, 3234, 3236, 2218, 690, 1207, 1211, 3269, 2767, 1761, 3818, 2283, 1260, 757, 1287, 3338, 2834, 3355, 1314, 1841, 307, 821, 309, 3385, 838, 3410, 339, 349, 3422, 1396, 2949, 1414, 2953, 3978, 396, 2444, 3984, 1945, 1440, 942, 3503, 4019, 4023, 4024, 1476, 3526, 475, 2523, 3035, 1504, 1512, 489, 3563, 3564, 3577]
[3584, 3975, 2056, 2574, 146, 786, 2460, 160, 34, 164, 676, 3877, 677, 2088, 1960, 3370, 2468, 1448, 687, 3120, 1840, 1970, 818, 1076, 566, 1767, 3260, 2752, 833, 3266, 2628, 455, 2120, 331, 77, 1746, 2003, 2898, 2515, 2006, 3031, 600, 855, 1494, 345, 2781, 990, 2527, 864, 224, 1117, 3940, 2405, 2149, 1511, 2280, 361, 743, 619, 1644, 2796, 1388, 1391, 624, 2544, 496, 2542, 627, 247, 2686]
147


In [12]:
unique_nodes100 = set(unique_nodes) - set([3448, 3851, 3370, 777,  2771, 132, 814, 24])
print(unique_nodes100)
influencers = [3448, 3851, 3370, 777,  2771, 132, 814, 24]
#maximize the score by trying the best nodes with 100 cost that did not make through the intersection of each centrality
max_score=0
all_120_200

for node100 in unique_nodes100:
    influencers.append(node100)
    #for node100 in unique_nodes100:
        # influencers.append(node100)
    purchased = set(influencers)
    print("Influencers: ", influencers)
    influencers_cost = get_influencers_cost(cost_path, influencers)
    #check if the cost  then 1000
    if influencers_cost > 1000:
        print("*************** Influencers are too expensive! ***************")
        exit()
    print("Influencers cost: ", influencers_cost)
    current_score = get_expectation_of_final_score(NoseBook_network, purchased)
    if current_score>max_score:
        max_score=current_score
        # max_node=node100
        max_node2=node100
    #influencers.remove(node100)
    influencers.remove(node100)    
print("*************** Your final score is " + str(max_score) + ", 200 or120_node " + str(max_node2) 
  )  #+", 200 or 120 node " )
print("Influencers: ", influencers+ [max_node2])




{259, 525, 2576, 2583, 1817, 798, 291, 3112, 3115, 3892, 1589, 3125, 1080, 3642, 317, 318, 2623, 3391, 319, 2881, 3654, 3659, 81, 1110, 3926, 2394, 350, 1887, 3681, 872, 617, 2154, 1897, 2412, 367, 1651, 2679, 384, 3461, 135, 1421, 2191, 3731, 3995, 2972, 932, 2217, 1453, 1455, 2480, 1719, 3002, 449, 707, 2245, 2503, 200, 1483, 3792, 2516, 3798, 729, 3802, 3293, 229, 3557, 3303, 3047, 2282, 746, 1259, 1264, 3318, 2295, 1274, 511}
Influencers:  [3448, 3851, 3370, 777, 2771, 132, 814, 24, 259]
Influencers cost:  1000
*************** Expected final score is 1803.78 ***************
Influencers:  [3448, 3851, 3370, 777, 2771, 132, 814, 24, 525]
Influencers cost:  1000
*************** Expected final score is 1784.51 ***************
Influencers:  [3448, 3851, 3370, 777, 2771, 132, 814, 24, 2576]
Influencers cost:  1000
*************** Expected final score is 1791.7 ***************
Influencers:  [3448, 3851, 3370, 777, 2771, 132, 814, 24, 2583]
Influencers cost:  1000
*************** Expected 